In [ ]:
# add a columns to processed_df to max the folliwing outcomes "Grant Revoked", "Judgment Delivered", "Judgment Delivered- Acquittal", "Judgment Delivered- Case Closed", "Judgment Delivered- Convicted" 
processed_df['max_resolved'] = processed_df.groupby('case_number')['concluded'].transform('max')

In [ ]:
def determine_judgment_scheduling(df):
    judgment_date_set_outcomes = ["Judgment Date Given", "Judgment On Notice", "Judgment Date Set"]
    judgment_delivered_outcomes = ["Grant Revoked", "Judgment Delivered", 
                                   "Judgment Delivered- Acquittal", "Judgment Delivered- Case Closed", 
                                   "Judgment Delivered- Convicted"]
    
    result = df.copy()
    
    result['first_scheduled_date'] = pd.NaT
    result['actual_judgment_date'] = pd.NaT
    result['was_adjourned'] = False
    result['judgment_delayed'] = False
    
    grouped = df.groupby('case_number')
    
    for case_numberanalyze_court_outcomes, group in grouped:
        group = group.sort_values('activity_date')
        
        judgment_set_rows = group[group['outcome'].isin(judgment_date_set_outcomes)]
        if not judgment_set_rows.empty:
            judgment_set_date = judgment_set_rows['activity_date'].min()
            first_scheduled = judgment_set_rows.loc[judgment_set_rows['activity_date'] == judgment_set_date, 'next_date'].iloc[0]
            
            result.loc[result['case_number'] == case_number, 'first_scheduled_date'] = first_scheduled
            
            next_activities = group[group['activity_date'] >= first_scheduled]
            if not next_activities.empty:
                next_activity = next_activities.iloc[0]
                if next_activity['outcome'] in judgment_delivered_outcomes:
                    result.loc[result['case_number'] == case_number, 'actual_judgment_date'] = next_activity['activity_date']
                elif next_activity['activity_date'] > first_scheduled:
                    result.loc[result['case_number'] == case_number, 'judgment_delayed'] = True
                    result.loc[result['case_number'] == case_number, 'was_adjourned'] = (next_activity['outcome'] == 'Matter Adjourned')
            
            if pd.isna(result.loc[result['case_number'] == case_number, 'actual_judgment_date'].iloc[0]):
                actual_judgment = group[group['outcome'].isin(judgment_delivered_outcomes)]
                if not actual_judgment.empty:
                    result.loc[result['case_number'] == case_number, 'actual_judgment_date'] = actual_judgment['activity_date'].max()
    
    return result

def determine_judgment_scheduling(df):
    # Define the outcomes representing judgment date set and judgment delivered
    judgment_date_set_outcomes = ["Judgment Date Given", "Judgment On Notice", "Judgment Date Set"]
    judgment_delivered_outcomes = ["Grant Revoked", "Judgment Delivered", 
                                   "Judgment Delivered- Acquittal", "Judgment Delivered- Case Closed", 
                                   "Judgment Delivered- Convicted"]
    
    # Create a copy of the original DataFrame
    result = df.copy()
    
    # Initialize new columns
    result['first_scheduled_date'] = pd.NaT
    result['actual_judgment_date'] = pd.NaT
    result['was_adjourned'] = False
    result['judgment_delayed'] = False
    
    # Group by case_number
    grouped = df.groupby('case_number')
    
    for case_number, group in grouped:
        # Sort the group by activity_date
        group = group.sort_values('activity_date')
        
        # Find the first scheduled judgment date
        judgment_set_rows = group[group['outcome'].isin(judgment_date_set_outcomes)]
        if not judgment_set_rows.empty:
            first_scheduled = judgment_set_rows['activity_date'].min()
            next_date = judgment_set_rows.loc[judgment_set_rows['activity_date'] == first_scheduled, 'next_date'].iloc[0]
            
            result.loc[result['case_number'] == case_number, 'first_scheduled_date'] = first_scheduled
            
            # Check if the next activity date matches the next_date and if judgment was delivered
            next_activity = group[group['activity_date'] > first_scheduled].iloc[0] if not group[group['activity_date'] > first_scheduled].empty else None
            
            if next_activity is not None:
                if next_activity['activity_date'] == next_date:
                    if next_activity['outcome'] in judgment_delivered_outcomes:
                        result.loc[result['case_number'] == case_number, 'actual_judgment_date'] = next_activity['activity_date']
                    else:
                        result.loc[result['case_number'] == case_number, 'was_adjourned'] = True
                else:
                    result.loc[result['case_number'] == case_number, 'judgment_delayed'] = True
            
            # If judgment wasn't delivered on the next_date, find when it was actually delivered
            if pd.isna(result.loc[result['case_number'] == case_number, 'actual_judgment_date'].iloc[0]):
                actual_judgment = group[group['outcome'].isin(judgment_delivered_outcomes)]
                if not actual_judgment.empty:
                    result.loc[result['case_number'] == case_number, 'actual_judgment_date'] = actual_judgment['activity_date'].max()
    
    return result

In [ ]:

# Define the priority order
priority_order = [
    'Judgment Delivered- Case Closed',
    'Judgment Delivered',
    'Judgment Delivered- Acquittal',
    'Judgment Delivered- Convicted',
    'Grant Revoked',
    'Ruling Delivered- Case Closed',
    'Ruling Delivered- Accused Discharged'
]

# Create a priority mapping for efficient lookups
priority_mapping = {value: index for index, value in enumerate(priority_order)}

def get_resolved_cases_by_court(df, outcome_col='outcome', court_col='court'):
    # Assign priority to each row based on the outcome
    df['priority'] = df[outcome_col].map(priority_mapping)
    
    # Filter rows with resolved cases
    resolved_cases = df[df['priority'].notna()]
    
    # Drop duplicates to ensure one outcome per case with the highest priority
    resolved_cases = resolved_cases.sort_values(by='priority').drop_duplicates(subset=['case_number', court_col], keep='first')
    
    # Drop the priority column as it is no longer needed
    resolved_cases = resolved_cases.drop(columns=['priority'])
    
    # Pivot the table
    pivot_table = resolved_cases.pivot_table(index=court_col, columns=outcome_col, values='case_number', aggfunc='count', fill_value=0)
    
    return pivot_table

# Example usage
# df = pd.DataFrame({...})
# result = get_resolved_cases_by_court(df)
# print(result)


In [ ]:
def calculate_judgment_time(df):
    # Define the outcomes representing judgment date set and judgment delivered
    judgment_date_set_outcomes = ["Judgment Date Given", "Judgment On Notice", "Judgment Date Set"]
    judgment_delivered_outcomes = ["Grant Revoked", "Judgment Delivered", 
                                   "Judgment Delivered- Acquittal", "Judgment Delivered- Case Closed", 
                                   "Judgment Delivered- Convicted"]

    # Create a copy of the original DataFrame to avoid modifying it
    result = df.copy()

    # Initialize columns for set_date, delivery_date, and time_taken_days
    result['set_date'] = pd.NaT
    result['delivery_date'] = pd.NaT
    result['time_taken_days'] = pd.NaT

    # Group by case_number
    grouped = df.groupby('case_number')

    for case_number, group in grouped:
        # Find the minimum judgment set date
        set_date = group[group['outcome'].isin(judgment_date_set_outcomes)]['activity_date'].min()
        
        # Find the maximum judgment delivered date
        delivery_date = group[group['outcome'].isin(judgment_delivered_outcomes)]['activity_date'].max()

        # If there is no set date but there is a delivery date and an earliest "Judgement" entry
        earliest_judgement_date = group[group['comingfor'] == "Judgement"]['activity_date'].min()
        if pd.isna(set_date) and pd.notna(delivery_date) and pd.notna(earliest_judgement_date):
            # Get dates for the same case before the earliest "Judgement"
            previous_dates = group[(group['activity_date'] < earliest_judgement_date) & (group['activity_date'] >= group['activity_date'].min())]['activity_date']

            if not previous_dates.empty:
                set_date = previous_dates.max()

        # If both dates are available, calculate the time taken
        if pd.notna(set_date) and pd.notna(delivery_date):
            time_taken_days = (delivery_date - set_date).days
            result.loc[result['case_number'] == case_number, 'set_date'] = set_date
            result.loc[result['case_number'] == case_number, 'delivery_date'] = delivery_date
            result.loc[result['case_number'] == case_number, 'time_taken_days'] = time_taken_days

    return result


In [ ]:
def get_on_time_delivery_proportions(scheduled_cases):
    # Get the final status for each case
    final_status = scheduled_cases.groupby(['court', 'case_number']).last().reset_index()
    
    # Group by court and calculate statistics
    court_stats = final_status.groupby('court').agg({
        'case_number': 'count',
        'delivery_category': lambda x: (x == 'On Time').sum()
    }).rename(columns={
        'case_number': 'total_scheduled',
        'delivery_category': 'delivered_on_time'
    })
    
    # Calculate the proportion
    court_stats['proportion_on_time'] = court_stats['delivered_on_time'] / court_stats['total_scheduled']
    
    return court_stats

In [ ]:
def determine_scheduled_judgment(df, cutoff_date):
    judgment_date_set_outcomes = ["Judgment Date Given", "Judgment On Notice", "Judgment Date Set"]
    judgment_delivered_outcomes = ["Grant Revoked", "Judgment Delivered", 
                                   "Judgment Delivered- Acquittal", "Judgment Delivered- Case Closed", 
                                   "Judgment Delivered- Convicted"]
    
    df['judgment_status'] = 'Not Scheduled'
    df['set_date'] = pd.NaT
    df['delivery_date'] = pd.NaT
    df['delivery_category'] = ''
    
    # Filter rows with judgment set outcomes
    judgment_set_rows = df[df['outcome'].isin(judgment_date_set_outcomes)]
    
    # Only consider rows with a scheduled date before or equal to the cutoff date
    valid_scheduled = judgment_set_rows[judgment_set_rows['next_date'] <= cutoff_date]
    
    # For each valid schedule, find the earliest set date
    earliest_schedule = valid_scheduled.sort_values('activity_date').groupby('case_number').first().reset_index()
    
    # Create a dictionary to map case numbers to their schedule dates and statuses
    case_to_set_date = dict(zip(earliest_schedule['case_number'], earliest_schedule['next_date']))
    case_to_status = {case: 'Scheduled' for case in earliest_schedule['case_number']}
    
    # Update the result dataframe with schedule information
    df['set_date'] = df['case_number'].map(case_to_set_date)
    df['judgment_status'] = df['case_number'].map(case_to_status).fillna('Not Scheduled')
    df['delivery_category'] = df['case_number'].map(case_to_status).fillna('')
    
    # Filter rows with judgment delivered outcomes
    judgment_delivered_rows = df[df['outcome'].isin(judgment_delivered_outcomes)]
    
    # Find the first delivery date after set date
    for case_number, group in earliest_schedule.groupby('case_number'):
        set_date = group['next_date'].values[0]
        delivery = judgment_delivered_rows[(judgment_delivered_rows['case_number'] == case_number) & 
                                           (judgment_delivered_rows['activity_date'] >= set_date)]
        if not delivery.empty:
            delivery_date = delivery.sort_values('activity_date').iloc[0]['activity_date']
            df.loc[df['case_number'] == case_number, 'delivery_date'] = delivery_date
            df.loc[df['case_number'] == case_number, 'judgment_status'] = 'Delivered'
            
            if delivery_date <= set_date:
                df.loc[df['case_number'] == case_number, 'delivery_category'] = 'On Time'
            else:
                df.loc[df['case_number'] == case_number, 'delivery_category'] = 'Delayed'
        else:
            if cutoff_date >= set_date:
                df.loc[df['case_number'] == case_number, 'judgment_status'] = 'Delayed'
                df.loc[df['case_number'] == case_number, 'delivery_category'] = 'Delayed'
    
    return df[df['set_date'].notna()]